In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import wosis
import pandas as pd
import metaknowledge as mk

In [ ]:
wos_config = wosis.load_config("config.yml")

In [ ]:
# dollar sign ($) is a wildcard used to match British and American spellings
# e.g. flavo$r matches both flavour and flavor
# see https://images.webofknowledge.com/images/help/WOS/hs_wildcards.html
inclusive_search_terms = ("Integrated Assessment and Model$ing", "integrated environmental model$ing", 
                          "integrated assessment", "integrated model$ing", "environmental model$ing", 
                          "hydrological model$ing", "ecological model$ing", "ecosystem model$ing", "model$ing framework",
                           "framework development", "model integration")
exclusive_search_terms = ("chemical", "industry", "cancer", "gene")
subject_area = ("ENVIRONMENTAL SCIENCES", "WATER RESOURCES", "ENGINEERING ENVIRONMENTAL", "INTERDISCIPLINARY APPLICATIONS")

In [ ]:
topics = [wosis.build_query(inclusive_search_terms, exclusive_search_terms, subject_area), ]

In [ ]:
topics

See [field tags](http://ipscience-help.thomsonreuters.com/wosWebServicesLite/WebServiceOperationsGroup/WebServiceOperations/g2/user_query/field_tags/WOSfieldTags.html) to help construct search parameters
    

## Send query to Clarivate Web of Science servers

In [ ]:
%%time
overwrite = False
id_to_query = wosis.query(topics, overwrite, wos_config)

In [ ]:
import json

with open('tmp/hash_to_query.txt', 'w') as file:
     file.write(json.dumps(id_to_query, indent=2))  # use `json.loads` to do the reverse

In [ ]:
query_id = list(id_to_query.keys())[0]

In [ ]:
RC = mk.RecordCollection(f"tmp/{query_id}.txt")
RC = RC.yearSplit(0, 2017)
print("Corpora consists of", len(RC), "Publications")

# Create representative data store

Dataset for publication cannot include WoS generated keywords or abstract text. It must also include attribution text.

In [ ]:
from datetime import datetime

corpora = RC.forNLP(extraColumns=["AU", "SO", "DE"])
corpora_df = pd.DataFrame(corpora)
corpora_df.drop(["keywords", "abstract"], inplace=True, axis=1)

corpora_fn = f"data/{query_id}.csv"
with open(corpora_fn, 'w') as fn:
    fn.write("# Data from Clarivate Analytics' Web of Science, retrieved 29 October 2018\n")
    fn.write("# This file generated on {}\n".format(datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
    corpora_df.index.name = "item"
    corpora_df.to_csv(fn)